In [1]:
import pandas as pd
from selenium import webdriver
import requests
import os

In [2]:
DRIVER_PATH = 'C:/Users/simon/Jupyter/chromedriver'  ### insert path to webdriver
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

## Collect the URLs of recent offers

In [3]:
overview_url = 'https://www.chrono24.com/search/index.htm?SEARCHSTATS_BRAND_ID=&SEARCHSTATS_MATERIAL_ID=&STARTSEARCH=&braceletMaterial=&caseMaterials=&countryIds=&dosearch=true&gender=&maxAgeInDays=&pageSize=120&priceFrom=&priceTo=&query=&resultview=list&searchexplain=false&showpage={page}&sortorder=5&usedOrNew=new&usedOrNew=used&watchCategories=&watchTypes=U'.format(page = 1)
driver.get(overview_url)

In [4]:
### Get the last 100 pages worth of new offers. One page contains 120 offers.
n_pages = 100
url_list = []
for i in range(1, n_pages+1):
    overview_url = 'https://www.chrono24.com/search/index.htm?SEARCHSTATS_BRAND_ID=&SEARCHSTATS_MATERIAL_ID=&STARTSEARCH=&braceletMaterial=&caseMaterials=&countryIds=&dosearch=true&gender=&maxAgeInDays=&pageSize=120&priceFrom=&priceTo=&query=&resultview=list&searchexplain=false&showpage={page}&sortorder=5&usedOrNew=new&usedOrNew=used&watchCategories=&watchTypes=U'.format(page = i)
    driver.get(overview_url)
    
    elems = driver.find_elements_by_xpath("//div[contains(@class, 'article-item-container wt-search-result')]")
    
    for elem in elems:
        url = elem.find_element_by_xpath(".//a[@href]").get_attribute("href")
        url_list.append(url)

In [5]:
## Although rare, sometimes incorrect links are collected.
## All URLs that dont contain the pattern 'id12345678' are removed.

def is_product(url):
    if(url[-12:-5].isdigit() and url[-14:-12] == 'id'):
        return True
    else:
        return False
    
url_list = [i for i in url_list if is_product(i)]

In [7]:
print(len(url_list))

12001


In [8]:
## this function takes an URL and returns the product ID, which are the last 8 digits before the '.htm' ending.
def get_id(url):
    i = url[-12:-4]
    return i

In [9]:
## this function downloads a product image and saves it under the product ID
def get_img(url, watch_id):
    img = requests.get(url).content
    path = 'images/{i}.jpg'.format(i=watch_id)
    
    with open(path, 'wb') as handler:
        handler.write(img)

In [10]:
## this function collects all the relevant product data and returns a dictionary
def get_data(url):
    driver.get(url)

    ##### Test if url is still the same
    ##### return NA or something if not
    if (driver.current_url != url):
        return ''
    
    watch_id = get_id(url)
    
    ### open table if collapsed
    if driver.find_elements_by_xpath("//button[@class='js-expand-button wt-expand-button text-link p-a-0']"):
        if(driver.find_element_by_xpath("//button[@class='js-expand-button wt-expand-button text-link p-a-0']").is_displayed()):
            driver.find_element_by_xpath("//button[@class='js-expand-button wt-expand-button text-link p-a-0']").click()
            
    ### Grab url if there is an image
    if driver.find_elements_by_xpath("//div[@class='jq-zoom-image c24-zoom-0 c24-zoom']"):
        img = driver.find_element_by_xpath("//div[@class='jq-zoom-image c24-zoom-0 c24-zoom']").get_attribute('data-original')
        get_img(img, watch_id)
    else:
        img = ''

    ## get the price
    price = driver.find_element_by_xpath("//div[@class='detail-page-price wt-detail-page-price']/span[@class='price-lg']").text
    ## is it a private seller?
    private = 'Private Seller' in driver.find_element_by_xpath("//section[@id='jq-seller']").text

    

    ## get all the product information
    spec_items = ['id', 'price', 'url', 'img_url', 'private_seller']
    spec_values = [watch_id, price, url, img, private]
    
    table = driver.find_element_by_xpath("//section[@id='jq-specifications']")
    tbodies = table.find_elements_by_xpath(".//tbody")
    
    for tb in tbodies:
        rows = tb.find_elements_by_xpath(".//tr")
        section = rows[0].text
        rows.pop(0)
        for row in rows:
            elems = row.find_elements_by_xpath(".//td")

            if(len(elems) > 1):
                spec_items.append(elems[0].text)
                spec_values.append(elems[1].text)
            else:
                spec_items.append(section)
                spec_values.append(elems[0].text)
            
    
    dictionary = dict(zip(spec_items, spec_values))
    return(dictionary)

In [11]:
if os.path.isfile('data.csv'):
    df = pd.read_csv('data.csv')
else:
    df = pd.DataFrame(columns=['url'])

print(len(df))
df.head()

58100


C:\Users\simon\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,Others,Power reserve,Base caliber,Lug width,Movement/Caliber,Number of jewels,Bracelet length,Buckle width,Frequency,Bracelet thickness
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for url in url_list:
    if(url not in df.url.values):
        print(url)
        data = get_data(url)

        if(data != ''):
            df = df.append(data, ignore_index=True)

https://www.chrono24.com/harrywinston/harry-winston-avenue-rose-gold-mop--id20299872.htm
https://www.chrono24.com/longines/longines-la-grande-classique--id20299854.htm
https://www.chrono24.com/breitling/breitling-superocean-automatic-44-a17367d71b1s1--id20299849.htm
https://www.chrono24.com/longines/longines-la-grande-classique-de-longines-quartz-blue--id20299845.htm
https://www.chrono24.com/breitling/breitling-superocean-automatic-44-a17367d71b1a1--id20299835.htm
https://www.chrono24.com/rolex/rolex-gmt-master-long-e-red-back-bezel--id20299831.htm
https://www.chrono24.com/longines/longines-la-grande-classique--id20299830.htm
https://www.chrono24.com/rolex/rolex-yacht-master-ii-ref-116688-ungetragen-aus-2019--id20299824.htm
https://www.chrono24.com/longines/longines-la-grande-classique--id20299822.htm
https://www.chrono24.com/patekphilippe/patek-philippe-calatrava-manual-winding-small-seconds-white-gold-37mm-5196g-001--id20299820.htm
no image
https://www.chrono24.com/breitling/breitlin

https://www.chrono24.com/tagheuer/formula-1-chronograph-steel-quartz-watch-cau1115--id20299446.htm
https://www.chrono24.com/omega/seamaster-professional-18k-gold-steel-watch-3961042--id20299445.htm
https://www.chrono24.com/omega/seamaster-automatic-stainless-steel-mens-dress-watch-166010--id20299444.htm
https://www.chrono24.com/omega/seamaster-automatic-stainless-steel-mens-dress-watch-2849--id20299443.htm
https://www.chrono24.com/tagheuer/aquaracer-stainless-steel-quartz-mens-watch-wap1112--id20299442.htm
https://www.chrono24.com/tiffany/atlas-quartz-yellow-gold-18k-womens-dress-watch-d286753--id20299441.htm
https://www.chrono24.com/omega/seamaster-quartz-stainless-steel-womens-sports-watch-258220--id20299440.htm
https://www.chrono24.com/tagheuer/link-chronograph-steel-automatic-mens-watch-cj1110--id20299439.htm
https://www.chrono24.com/bulgari/automatic-stainless-steel-mens-dress-watch-bb33slauto--id20299438.htm
https://www.chrono24.com/tagheuer/aquaracer-quartz-gold-platedstainless-

https://www.chrono24.com/carlfbucherer/carl-f-bucherer-adamavi-39-i-factory-diamonds-i-rosegold-750--id20299061.htm
https://www.chrono24.com/rolex/rolex-submariner--hulk-steel-116610lv--id20299052.htm
https://www.chrono24.com/tudor/tudor-black-bay--id20299048.htm
https://www.chrono24.com/breitling/breitling-avenger-hurricane-45-xb0180e4-2020--id20299019.htm
https://www.chrono24.com/breitling/breitling-endurance-pro-ironman--id20299017.htm
https://www.chrono24.com/revuethommen/revue-thommen-heritage--id20299006.htm
https://www.chrono24.com/tagheuer/tag-heuer-carrera-gmt-re-edition-1964--id20298997.htm
no image
https://www.chrono24.com/vacheronconstantin/vacheron-constantin-patrimony-18k-rose-gold-85180000r-9166--id20298988.htm
https://www.chrono24.com/cartier/cartier-watch-panthere-18k-quartz--id20298983.htm
https://www.chrono24.com/breitling/breitling-montbrillant-legende-c23340-box--papers--id20298981.htm
https://www.chrono24.com/tagheuer/tag-heuer-formula-1--id20298978.htm
https://ww

https://www.chrono24.com/hublot/hublot-classic--id20298376.htm
https://www.chrono24.com/graham/graham-silverstone-tourbillograph--id20298368.htm
https://www.chrono24.com/cartier/cartier-tank-solo-xl--id20298366.htm
https://www.chrono24.com/breitling/breitling-top-time--id20298332.htm
https://www.chrono24.com/seiko/seiko-sportsman-17--id20298329.htm
https://www.chrono24.com/rolex/datejust-36-jubilee-silver-diamond-dial--id20298323.htm
https://www.chrono24.com/breguet/breguet-marine--id20298309.htm
https://www.chrono24.com/omega/speedmaster-moonwatch-31130423001005-black-dial-hand-wind-mens-watch--id20298308.htm
https://www.chrono24.com/omega/de-ville-tresor-master-co-axial-43253402102001-silver-dial-manual-win--id20298307.htm
https://www.chrono24.com/tagheuer/tag-heuer-chronometer-200m-automatic--id20298294.htm
https://www.chrono24.com/rolex/rolex-explorer-2021-36mm--id20298276.htm
https://www.chrono24.com/all/d1-milano-pr06-polycarbon-herren-40mm-5m--id20298269.htm
https://www.chrono24

https://www.chrono24.com/cartier/tortue-paris-2496-yellow-gold-diamonds-34x43mm--id20298030.htm
https://www.chrono24.com/rolex/day-date-118296-platinum-diamonds-36mm--id20298029.htm
https://www.chrono24.com/girardperregaux/ultra-thin-manual-winding-yellow-gold-33mm--id20298028.htm
https://www.chrono24.com/piaget/1980s-lady-vintage-in-yellow-gold-27mmx24mm---61-grams--id20298027.htm
https://www.chrono24.com/cartier/santos-ronde-w3315-in-18kt-yellow-gold--id20298026.htm
https://www.chrono24.com/cartier/lanieres-mini-tonneau-12072ce-in-18kt-white-gold--id20298025.htm
https://www.chrono24.com/longines/vintage-yellow-gold-30mm-x-31mm--id20298024.htm
https://www.chrono24.com/tudor/black-bay-bronze-blue-79250bb-43mm-limited-edition--id20298023.htm
https://www.chrono24.com/franckmuller/black-magic-limited-edizion-400pz-902qz-d-white-gold-pvd-leather--id20298022.htm
https://www.chrono24.com/hublot/tutti-frutti-black-apple-big-bang-federacion-mexicana---de-futbol-asoc-ac-ceramic-41mm--id20298021

...

https://www.chrono24.com/stowa/seatime-pro-diver-lime-green-42mm-swiss-automatic-1000m-blasted-titanium--id20265626.htm
https://www.chrono24.com/phantoms/new-phantoms-lab-apocalyptic-force-flying-tourbillon-limited-edition-manual-wind--id20265625.htm
https://www.chrono24.com/all/jorg-schauer-kleine-schauer-15th-anniversary-one-hand-limited-edition-50-pieces--id20265624.htm
https://www.chrono24.com/sector/sharkmaster-1000-swiss-eta-2824-automatic-1000m-diver-46mm-black-pvd--id20265623.htm
https://www.chrono24.com/all/deutsche-master-1936-air-force-regulateur-limited-edition-47mm--id20265621.htm
https://www.chrono24.com/audemarspiguet/royal-oak--id20265611.htm
https://www.chrono24.com/fortis/spacematic--id20265610.htm
https://www.chrono24.com/omega/omega-geneve--id20265609.htm
https://www.chrono24.com/rolex/rolex-gmt-master-ii--bestzustand-aus-09-2014---id20265605.htm
https://www.chrono24.com/rolex/rolex-rolex-datejust-stahlweissgold-36mm-aus-072021-neu--id20265602.htm
https://www.chrono

https://www.chrono24.com/certina/certina-vintage-ds-2-super-ph500m-tektite--id20265224.htm
https://www.chrono24.com/tagheuer/stainless-steel-aquaracer-way2110ba0928--id20265223.htm
https://www.chrono24.com/panerai/luminor-marina-last-one-for-paneristi--excellent-full-set--pam00911-pam911--id20265218.htm
https://www.chrono24.com/breguet/breguet-reine-de-naples--id20265205.htm
https://www.chrono24.com/hublot/hublot-classic-fusion-42-mm--id20265197.htm
https://www.chrono24.com/rolex/rolex-explorer-ii-216570-white-new-unwornapr21fullsetreceipt-opt--id20265191.htm
https://www.chrono24.com/citizen/divers-professionale-certificato-iso-6425-bn4021-02e-promaster-eco-drive-super-titanop--id20265185.htm
https://www.chrono24.com/patekphilippe/patek-philippe-ellipse-ref-3548-manual-winding-gold-18kt--id20265183.htm
https://www.chrono24.com/rolex/rolex-datejust-36-oystersteel--weissgold-126234-schiefergrau-roemisch-jubile--id20265180.htm
https://www.chrono24.com/rolex/datejust-28-18k-rose-gold-amp-s

https://www.chrono24.com/cuervoysobrinos/cuervo-y-sobrinos-3191vas-historiador-pequenis-segundos-watch--id20264820.htm
https://www.chrono24.com/rolex/rolex-rolex-oyster-perpetual-datejust-black-16014-1984--id20264818.htm
https://www.chrono24.com/cartier/cartier-santos-galbee-24mm-automatic-steel-watch-065tcw-diamond-bezel-new-model--id20264811.htm
https://www.chrono24.com/tagheuer/aquaracer-300m--id20264805.htm
https://www.chrono24.com/vancleefarpels/van-cleef--arpels-la-collection-43607-serviced--id20264803.htm
https://www.chrono24.com/breitling/emergency--id20264802.htm
https://www.chrono24.com/rolex/rolex-yacht-master-40-126622-rhodium-unworn--id20264791.htm
https://www.chrono24.com/tagheuer/aquaracer-300m-stahlband--original-rubberstrap---garantie-bis-052024--id20264756.htm
https://www.chrono24.com/rolex/rolex-rolex-oyster-perpetual-datejust-black-15200-2000-full-set--id20264746.htm
https://www.chrono24.com/rolex/rolex-datejust-41-126334-steel--white-gold-azzurro-blue-roman-dial-bp

https://www.chrono24.com/baumemercier/classima-quartz-silver-dial-brown-leather-strap-mens-watch-10415--id20264440.htm
https://www.chrono24.com/ulyssenardin/marine-chronometer-manufacture-1183-122-342-v2--id20264434.htm
https://www.chrono24.com/tagheuer/tag-heuer-carrera-war1315ba0778--id20264432.htm
https://www.chrono24.com/omega/constellation--id20264431.htm
https://www.chrono24.com/rolex/rolex-oyster-perpetual-36-silver-dial-126000-unworn-062021--id20264429.htm
https://www.chrono24.com/seiko/1970-divers-re-creation-sla033-new-and-full-set--id20264423.htm
https://www.chrono24.com/rado/starliner-vintage-1960-swiss--id20264417.htm
https://www.chrono24.com/carlfbucherer/patravi--id20264415.htm
https://www.chrono24.com/omega/omega-seamaster-120-tropic-strap-serviced--id20264413.htm
https://www.chrono24.com/rolex/daytona-silver-panda-bj2021--id20264410.htm
https://www.chrono24.com/baumemercier/baume--mercier-clifton-10518--id20264407.htm
https://www.chrono24.com/omega/omega-speedmaster-38

https://www.chrono24.com/omega/omega-speedmaster-reduced-automatic-full-set-serviced--id20264086.htm
https://www.chrono24.com/omega/omega-speedmaster-moonwatch-co-axial-master-chronometer--id20264084.htm
https://www.chrono24.com/tagheuer/formula-1-calibre-5-automatik-43mm-waz2011ba0843--id20264080.htm
https://www.chrono24.com/twsteel/wrc-limited-editions-new-boys-in-town-wrc-limited-editions-gt11--id20264078.htm
https://www.chrono24.com/hamilton/american-classic-intra-matic-68-autochrono-panda-h38416711--id20264077.htm
https://www.chrono24.com/twsteel/ceo-tech-44mm-ce4045--id20264076.htm
https://www.chrono24.com/twsteel/ceo-tech-44mm-ce4047--id20264075.htm
https://www.chrono24.com/twsteel/volante-svs201--id20264071.htm
https://www.chrono24.com/rolex/rolex-gmt-master-ii-40mm-rootbeer-steel-rose-gold-126711-chnr-new-2021--id20264057.htm
https://www.chrono24.com/rolex/rolex-daytona--id20264053.htm
https://www.chrono24.com/casio/casio-casio-g-shock-mrg-g1000d-1ajr-solar-quartz-mens-watch--

In [20]:
print(len(df))
df.tail()

58100


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,Others,Power reserve,Base caliber,Lug width,Movement/Caliber,Number of jewels,Bracelet length,Buckle width,Frequency,Bracelet thickness
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
if os.path.isfile('data.csv'):
    os.remove('data_backup.csv')
    os.rename('data.csv', 'data_backup.csv')

df.to_csv('data.csv')